<a href="https://colab.research.google.com/github/smrutipunto/DS-for-argri/blob/main/DS_Agri_4_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Practical 4 Analyzing Satellite Images for Crop Health Monitoring: Use satellite imagery to assess crop health and identify areas needing attention. Tools: QGIS, Python, OpenCV.
#pracrical 4
# Analyzing Satellite Images for Crop Health Monitoring: Use
satellite imagery to assess crop health and identify areas needing
attention. Tools: QGIS, Python, OpenCV
var r=ee.ImageCollection("projects/neon-prodearthengine/assets/HSI_REFL/001");
print('Neon Directional Reflectance
Images:',r.aggregate_array('system:index'));

var startdate=ee.Date('2017-01-01');
var enddate=startdate.advance(1,'year');
var r_2017=r.filterDate(startdate,enddate);
var r_grsm_2017=r_2017.filter(ee.Filter.eq('NEON_SITE',
'GRSM')).mosaic();
var rgbvis={
 min:340,
 max:2150,
 bands: ['B053','B035','B019'],
 gamma:2
};
Map.addLayer(r_grsm_2017,rgbvis,'Reflectance RGB Image');
var nirband=r_grsm_2017.select('B170');
var redband=r_grsm_2017.select('B050');
var
ndvi=nirband.subtract(redband).divide(nirband.add(redband)).rename('
NDVI');
Map.addLayer(ndvi,{min:-
1,max:1,palette:['blue','white','green']},'NDVI')
var hv=ndvi.gt(0.8).selfMask();
var sv=ndvi.lt(0.6).selfMask();
Map.addLayer(hv,{min:-1,max:1,palette:['green']},'High Vegetation')
Map.addLayer(sv,{min:-1,max:1,palette:['red']},'Stressed
Vegetation')
Map.setCenter(-83.5, 35.65, 14);

In [1]:
# Practical 5
# Land Use Classification Using Remote Sensing Data: Classify land
use patterns using remote sensing data and mac
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
# Load the EuroSAT dataset.
# The dataset contains 27,000+ labeled satellite images (64x64
pixels, 3 bands)
# It is widely used as a benchmark for land cover classification.
(ds_train, ds_test), ds_info = tfds.load(
 'eurosat',
 split=['train[:80%]', 'train[80%:]'],
 as_supervised=True, # returns tuple (image, label)
 with_info=True
)
# Preprocessing function to normalize image pixels to [0,1]
def preprocess(image, label):
 image = tf.cast(image, tf.float32) / 255.0
 return image, label
# Set up dataset pipelines: apply preprocessing, shuffle and batch.
BATCH_SIZE = 32
ds_train =
ds_train.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf
.data.AUTOTUNE)
ds_test =
ds_test.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
# Retrieve input shape and number of classes from ds_info.
input_shape = ds_info.features['image'].shape # e.g., (64, 64, 3)
num_classes = ds_info.features['label'].num_classes
# Retrieve the class names from the dataset info.
class_names = ds_info.features['label'].names
print("Class names:", class_names)
# Build a Convolutional Neural Network (CNN) using Keras.
model = tf.keras.Sequential([
 tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
input_shape=input_shape),
 tf.keras.layers.MaxPooling2D((2, 2)),
27
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(128, activation='relu'),
 tf.keras.layers.Dropout(0.5),
 tf.keras.layers.Dense(num_classes, activation='softmax')
])
# Compile the model with an optimizer, loss function, and accuracy
metric.
model.compile(optimizer='adam',
 loss='sparse_categorical_crossentropy',
 metrics=['accuracy'])
# Train the model.
EPOCHS = 10
history = model.fit(ds_train,
 epochs=EPOCHS,
 validation_data=ds_test)
# Evaluate the model on the test dataset.
test_loss, test_acc = model.evaluate(ds_test)
print("Test accuracy:", test_acc)
# -------------------------------
# Visualize Predictions on Test Images
# -------------------------------
# Take one batch from the test dataset
for images, labels in ds_test.take(1):
 predictions = model.predict(images)
 pred_labels = np.argmax(predictions, axis=1)
 true_labels = labels.numpy()
# Set up a grid for visualization.
num_images = 9 # number of images to display
plt.figure(figsize=(10, 10))
for i in range(num_images):
 ax = plt.subplot(3, 3, i + 1)
 plt.imshow(images[i].numpy())
 pred_class = class_names[pred_labels[i]]
 true_class = class_names[true_labels[i]]
 plt.title(f"True: {true_class}\nPred: {pred_class}")
 plt.axis("off")
28
plt.tight_layout()
plt.show()
Class names: ['AnnualCrop', 'Forest', 'HerbaceousVegetation',
'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential',
'River', 'SeaLake']

SyntaxError: unmatched ')' (<ipython-input-1-fdcd6c5f792d>, line 10)

In [ ]:
#Practical 6  Soil Moisture Prediction Using Sensor Data: Predict soil moisture levels using data from soil sensors and environmental factors.
# Soil Moisture Prediction Using Sensor Data: Predict soil moisture
levels using data from soil sensors and environmental factors.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dropout
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping
df = pd.read_csv('soil_data.csv')
# Data overview
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows of the dataset:")
print(df.head())
# Check for missing values
print("\nMissing values in each column:")
print(df.isnull().sum())
# Descriptive statistics
print("\nDescriptive statistics of the dataset:")
print(df.describe())
# Determine unique latitude-longitude pairs
unique_locations = df[['latitude',
'longitude']].drop_duplicates().reset_index(drop=True)
print(f"There are {unique_locations.shape[0]} unique locations in
the dataset.")
m = folium.Map(location=[51.1657, 10.4515], zoom_start=5,
tiles="OpenStreetMap", width=800, height=800)
# Add unique locations to the map
for idx, row in unique_locations.iterrows():
 folium.CircleMarker(
 location=(row['latitude'], row['longitude']),
 radius=2,
 color='blue',
 fill=True,
 fill_color='blue',
 fill_opacity=0.6
 ).add_to(m)
 # Display the map directly
m
      # Create a unique identifier for each location
df['location_id'] = df.groupby(['latitude', 'longitude']).ngroup()
33
# Determine the start and end of the time series
start_date = df['time'].min()
end_date = df['time'].max()
print(f"The time series starts on {start_date} and ends on
{end_date}.")
# Randomly select five unique location IDs
random_locations = np.random.choice(df['location_id'].unique(), 5,
replace=False)
# Plot time series data for the selected locations
plt.figure(figsize=(10, 5))
for location in random_locations:
 subset = df[df['location_id'] == location]
 lat = subset['latitude'].iloc[0]
 lon = subset['longitude'].iloc[0]
 plt.plot(pd.to_datetime(subset['time']), subset['sm_tgt'],
label=f"Location {location} (Lat: {lat:.2f}, Lon: {lon:.2f})")
plt.title("Soil Moisture Time Series for Randomly Selected
Locations")
plt.xlabel("Time")
plt.ylabel("Soil Moisture Value")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
# Set style and context to make the plot look fancy
34
sns.set_style("whitegrid")
sns.set_context("talk")
# Create the distribution plot
plt.figure(figsize=(12, 8))
sns.histplot(df['sm_tgt'], kde=True, bins=30, color='skyblue',
edgecolor='black', linewidth=1.2)
# Mark percentiles on the plot
percentiles = [5, 25, 50, 75, 95]
for percentile in percentiles:
 value = np.percentile(df['sm_tgt'], percentile)
 plt.axvline(value, color='red', linestyle='dashed', linewidth=1)
 plt.text(value, 5, f'{percentile}%', color='red', rotation=90,
verticalalignment='bottom')
# Set the title and labels
plt.title("Distribution of Soil Moisture for the Year 2013",
fontsize=18)
plt.xlabel("Soil Moisture Value", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.tight_layout()
plt.show()
# Randomly sample 2000 data points from the dataframe
sample_df = df.sample(n=2000, random_state=42)
# Set the aesthetic style of the plots
35
sns.set_style("whitegrid")
sns.set_context("talk")
# Initialize a figure with three subplots side by side
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 6))
# List of soil components
components = ['clay_content', 'sand_content', 'silt_content']
# Plot scatter plots for each component
for i, component in enumerate(components):
 sns.regplot(x=component, y='sm_tgt', data=sample_df, ax=axes[i],
color='skyblue', scatter_kws={'s':10}, line_kws={'color':'red'})

# Calculate correlation and annotate the plot with its value
 correlation = sample_df[component].corr(sample_df['sm_tgt'])
 axes[i].set_title(f"Correlation between
{component.split('_')[0].capitalize()} and Soil Moisture:
{correlation:.2f}", fontsize=14)
# Adjust layout for better display
plt.tight_layout()
plt.show()
# Convert the 'time' column to datetime format
df['time'] = pd.to_datetime(df['time'])
# Separate the data for June
june_data = df[df['time'].dt.month == 6]
# Remove June data from the main dataframe
df = df[df['time'].dt.month != 6]
june_data.head()
# Features to normalize
features_to_normalize = ['latitude', 'longitude', 'clay_content',
'sand_content', 'silt_content', 'sm_aux', 'location_id','sm_tgt']
# Initialize scaler
scaler = MinMaxScaler()
# Apply normalization to the dataframe (both training and June data)
df[features_to_normalize] =
scaler.fit_transform(df[features_to_normalize])
june_data[features_to_normalize] =
scaler.transform(june_data[features_to_normalize])
# Reorder the columns to ensure 'sm_tgt' is the last column
ordered_columns = ['time', 'latitude', 'longitude', 'clay_content',
'sand_content', 'silt_content', 'sm_aux', 'location_id', 'sm_tgt']
df = df[ordered_columns]
june_data = june_data[ordered_columns]
df.head()
june_data.head()
# Calculate the number of data points for each location in june_data
data_points_per_location = june_data.groupby('location_id').size()
# Identify locations with less than 15 days of data
locations_to_exclude =
data_points_per_location[data_points_per_location < 15].index
# Filter out these locations from june_data
june_data =
june_data[~june_data['location_id'].isin(locations_to_exclude)]
def create_sequences(data, seq_length):
 """
 Create input-output sequence pairs from the provided dataframe.
 """
 X, Y = [], []
 for location, group in data.groupby('location_id'):
 # Extract relevant features from the group
 features =
group[features_to_normalize].drop(columns='sm_tgt').values
 targets = group['sm_tgt'].values

 # Create sequences
 for i in range(len(features) - seq_length):
 X.append(features[i:i+seq_length])
 Y.append(targets[i+seq_length])

 return X, Y
# Define the sequence length (30 days)
seq_length = 15
# Create sequences for training data
X, Y = create_sequences(df, seq_length)
# Convert lists to numpy arrays for easier handling later
X = np.array(X)
Y = np.array(Y)
X.shape, Y.shape
from keras.models import Sequential
from keras.layers import LSTM, BatchNormalization, Dropout, Dense
from keras.regularizers import l2
model = Sequential()
model.add(LSTM(75, activation='relu', input_shape=(X.shape[1],
X.shape[2]), kernel_regularizer=l2(0.01),
recurrent_regularizer=l2(0.01), return_sequences=True))
model.add(LSTM(75, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
# Dense output layer
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=3,
verbose=1, restore_best_weights=True)
# Train the model with early stopping
history = model.fit(X, Y, epochs=10, batch_size=128,
validation_split=0.2, verbose=1, callbacks=[early_stop])
# Plot the training and validation loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('LSTM Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error (MSE)')
plt.legend()
plt.grid(True)
plt.show()
Code:
_june, Y_june_true = create_sequences(june_data, seq_length)
X_june = np.array(X_june);
Y_june_true = np.array(Y_june_true);
model.compile(optimizer='adam', loss='mse');
X_june.shape,Y_june_true.shape
predictions = model.predict(X_june,batch_size=128)
def reverse_scaling(data, scaler):
 """
 Reverse the scaling effect on data.
 """
 # Convert data to a 2D array if it's 1D
 if len(data.shape) == 1:
 data = data.reshape(-1, 1)

 dummy = np.zeros((len(data), len(features_to_normalize)))

 # Set the last column of the dummy array to your data
 dummy[:, -1] = data.ravel()

 # Use inverse_transform to reverse the scaling
 unscaled = scaler.inverse_transform(dummy)

 # Return the last column (our actual unscaled data)
 return unscaled[:, -1]
# Reverse scaling for Y_june_true and predictions
Y_june_true_unscaled = reverse_scaling(Y_june_true, scaler)
predictions_unscaled = reverse_scaling(predictions, scaler)
# Ensure both arrays are 1-dimensional
Y_june_true_unscaled = Y_june_true_unscaled.ravel()
predictions_unscaled = predictions_unscaled.ravel()
# Create a DataFrame for plotting
df_plot = pd.DataFrame({
 'Actual Values': Y_june_true_unscaled,
 'Predicted Values': predictions_unscaled
})
# Set Seaborn style
sns.set_style("whitegrid")
# Create a line plot
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_plot)
plt.title('Comparison of Actual vs Predicted Soil Moisture for
June')
plt.ylabel('Soil Moisture')
plt.xlabel('Days in June')
40
plt.legend()
plt.show()
# Calculate MAE
mae = np.mean(np.abs(Y_june_true_unscaled - predictions_unscaled))
print(f"Mean Absolute Error for June Predictions: {mae}")